In [3]:
# Embedding with llama3.2:3b
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

In [4]:
# --------------------
# Setup
# --------------------
os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")

persist_dir = "llama-emb"
embeddings = OllamaEmbeddings(model="llama3.2:3b")

# Load documents only if we need to create embeddings
if not os.path.exists(persist_dir) or not os.listdir(persist_dir):
    print("⚡ No existing DB found. Creating new embeddings...")

    # Load and split text
    loader = TextLoader("facts.txt")
    docs = loader.load()
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=200,
        chunk_overlap=50
    )
    docs = text_splitter.split_documents(docs)

    # Create Chroma DB and persist
    db = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_dir
    )
else:
    print("✅ Found existing DB. Loading without re-embedding...")
    db = Chroma(
        persist_directory=persist_dir,
        embedding_function=embeddings
    )

✅ Found existing DB. Loading without re-embedding...


In [5]:
# --------------------
# Run a test query
# --------------------
query = "What is an interesting fact about the English language?"
results = db.similarity_search(query, k=3)

for i, result in enumerate(results, start=1):
    print(f"\nResult {i}:")
    print(result.page_content)


Result 1:
89. The first TV commercial aired on July 1, 1941, for the Bulova Watch Company.
90. The modern Italian language originated from the region of Tuscany.

Result 2:
50. Canada has more lakes than the rest of the world combined.
51. 10% of the world's population is left-handed.
52. The Statue of Liberty was gifted to the U.S. by France.

Result 3:
84. Hedonic treadmill is a tendency of people to return to a set level of happiness despite life’s ups and downs.
85. Approximately two-thirds of people tip their head to the right when they kiss.
